# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project aims to building a data warehouse using the US immigration data set, enriching with the demographics of the US cities dataset and the world temperature data for data analysis. This is built for a data analytics table.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = None

### Step 1: Scope the Project and Gather Data

#### Project Scope 
In this project I will be integrating three data sets the immigration dataset, the temperature dataset and the US cities demographics dataset to create a data analytics table that can help answer questions like
 - immigrants move to US cities with what kind of demographics staistics?
 - immigrants move to US cities with what kind of temperature, do they move to warmer weather or cooler weather?
 - Do they move to cities with a larger population or less population?
 
 For this project I have used Pandas and numpy as tools. Pandas has great libraries for all the exploratory Data Analysis that I wanted to do and numpy I used for analysis on certain columns. But for future I reccomend using spark for analysis because it allows for distributed processing and also use cloud like S3 for storage, EMR and Redshift for analysis because for larger data more processing power is needed and using CPU doesnt allow for good processing. I have used .py files to create sparkifydb and tables on the localhost and I used Jupyter notebook to analyze the data and test for data quality. I used Jupyter notebook because it is easy and intuitive to use for analysis and creating the tables.



#### Describe and Gather Data 
Data Sources:

 - I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. https://www.trade.gov/national-travel-and-tourism-office
 - World Temperature Data: This dataset came from Kaggle. https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data
 - U.S. City Demographic Data: This data comes from OpenSoft. https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/


Let's take a look at the <strong>Immigration Table</strong>, explore, clean and break it into fact and dimension table

In [2]:
# Reading the i94 data
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immi = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
df_immi.head(10)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,57.0,1.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,63.0,2.0,1.0,20160401,NaN,NaN,O,K,NaN,M,1953.0,09302016,NaN,NaN,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,57.0,2.0,1.0,20160401,NaN,NaN,O,K,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,46.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1970.0,09302016,NaN,NaN,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,48.0,1.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1968.0,09302016,NaN,NaN,AZ,9.247849e+10,00608,B1


In [4]:
df_immi.shape

(3096313, 28)

In [5]:
df_immi.isnull().sum()

cicid             0
i94yr             0
i94mon            0
i94cit            0
i94res            0
i94port           0
arrdate           0
i94mode         239
i94addr      152372
depdate      142457
i94bir          802
i94visa           0
count             0
dtadfile          1
visapost    1881250
occup       3088187
entdepa         238
entdepd      138429
entdepu     3095921
matflag      138429
biryear         802
dtaddto         477
gender       414269
insnum      2982605
airline       83627
admnum            0
fltno         19549
visatype          0
dtype: int64

Since 'cicid', the primary key, is not null keeping the null's and filling it with 0.

In [6]:
df_immi.dtypes

cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port      object
arrdate     float64
i94mode     float64
i94addr      object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile     object
visapost     object
occup        object
entdepa      object
entdepd      object
entdepu      object
matflag      object
biryear     float64
dtaddto      object
gender       object
insnum       object
airline      object
admnum      float64
fltno        object
visatype     object
dtype: object

In [15]:
#df_immi = df_immi.fillna(0)

#df_immi = df_immi.astype({"i94bir": int, "i94yr": int, 'biryear': int,'i94mon': int,'i94visa': int})

#df_immi['depdate'] = pd.to_datetime(df_immi['depdate'],format='%d%m%Y').dt.date
#df_immi['depdate'] = pd.to_timedelta(df_immi['depdate'], unit='D') + pd.Timestamp('1960-1-1')

#df_immi['arrdate'] = pd.to_timedelta(df_immi['arrdate'], unit='D') + pd.Timestamp('1960-1-1')


#df_immi['Country'] = "United States"

In [7]:
def clean_df_immi(df_immi):
    df_immi = df_immi.fillna(0)
    df_immi = df_immi.astype({"i94bir": int, "i94yr": int, 'biryear': int,'i94mon': int,'i94visa': int})
    df_immi['depdate'] = pd.to_timedelta(df_immi['depdate'], unit='D') + pd.Timestamp('1960-1-1')
    df_immi['arrdate'] = pd.to_timedelta(df_immi['arrdate'], unit='D') + pd.Timestamp('1960-1-1')
    #Adding Country to the immigration dataframe to be able to merge with temperature table
    df_immi['Country'] = "United States"
    return df_immi

In [8]:
df_immi = clean_df_immi(df_immi)

In [9]:
df_immi.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,Country
0,6.0,2016,4,692.0,692.0,XXX,2016-04-29,0.0,0,1960-01-01,37,2,1.0,0,0,0,T,0,U,0,1979,10282016,0,0,0,1.897628e+09,0,B2,United States
1,7.0,2016,4,254.0,276.0,ATL,2016-04-07,1.0,AL,1960-01-01,25,3,1.0,20130811,SEO,0,G,0,Y,0,1991,D/S,M,0,0,3.736796e+09,00296,F1,United States
2,15.0,2016,4,101.0,101.0,WAS,2016-04-01,1.0,MI,2016-08-25,55,2,1.0,20160401,0,0,T,O,0,M,1961,09302016,M,0,OS,6.666432e+08,93,B2,United States
3,16.0,2016,4,101.0,101.0,NYC,2016-04-01,1.0,MA,2016-04-23,28,2,1.0,20160401,0,0,O,O,0,M,1988,09302016,0,0,AA,9.246846e+10,00199,B2,United States
4,17.0,2016,4,101.0,101.0,NYC,2016-04-01,1.0,MA,2016-04-23,4,2,1.0,20160401,0,0,O,O,0,M,2012,09302016,0,0,AA,9.246846e+10,00199,B2,United States


Seperating for the fact and dimensional tables. From the immigration table seperating it into fact_immigration, dim_personal_data and dim_airline_details

In [10]:
fact_immigration = df_immi[['cicid','i94yr','i94mon','i94port','arrdate','depdate','i94visa', 'i94mode','i94addr','Country']]

In [11]:
#renaming columns
fact_immigration.rename(columns={'i94yr': 'year', 'i94mon': 'month', 'i94port':'port','arrdate':'arrival_date', 'depdate':'departure_date','i94visa':'visa','i94mode':'mode','i94addr':'state_code'}, inplace=True)

In [12]:
dim_personal_data = df_immi[['cicid','i94cit', 'i94res','i94addr','i94bir','gender','insnum','biryear','occup','dtaddto' ]]

In [13]:
dim_personal_data.rename(columns={'i94cit': 'citizenship', 'i94res': 'residency','i94addr':'state_code','i94bir':'age','biryear':'birth_year','occup':'occupation','dtaddto':'admission_date'}, inplace=True)

In [14]:
dim_airline_details = df_immi[['cicid', 'airline','admnum','fltno','visatype']]

In [15]:
dim_airline_details.head(10)

,cicid,airline,admnum,fltno,visatype
0,6.0,0,1.897628e+09,0,B2
1,7.0,0,3.736796e+09,00296,F1
2,15.0,OS,6.666432e+08,93,B2
3,16.0,AA,9.246846e+10,00199,B2
4,17.0,AA,9.246846e+10,00199,B2
5,18.0,AZ,9.247104e+10,00602,B1
6,19.0,AZ,9.247140e+10,00602,B2
7,20.0,AZ,9.247161e+10,00602,B2
8,21.0,AZ,9.247080e+10,00602,B2
9,22.0,AZ,9.247849e+10,00608,B1


In [16]:
fact_immigration.head()

,cicid,year,month,port,arrival_date,departure_date,visa,mode,state_code,Country
0,6.0,2016,4,XXX,2016-04-29,1960-01-01,2,0.0,0,United States
1,7.0,2016,4,ATL,2016-04-07,1960-01-01,3,1.0,AL,United States
2,15.0,2016,4,WAS,2016-04-01,2016-08-25,2,1.0,MI,United States
3,16.0,2016,4,NYC,2016-04-01,2016-04-23,2,1.0,MA,United States
4,17.0,2016,4,NYC,2016-04-01,2016-04-23,2,1.0,MA,United States


In [17]:
dim_personal_data.head(10)

,cicid,citizenship,residency,state_code,age,gender,insnum,birth_year,occupation,admission_date
0,6.0,692.0,692.0,0,37,0,0,1979,0,10282016
1,7.0,254.0,276.0,AL,25,M,0,1991,0,D/S
2,15.0,101.0,101.0,MI,55,M,0,1961,0,09302016
3,16.0,101.0,101.0,MA,28,0,0,1988,0,09302016
4,17.0,101.0,101.0,MA,4,0,0,2012,0,09302016
5,18.0,101.0,101.0,MI,57,0,0,1959,0,09302016
6,19.0,101.0,101.0,NJ,63,0,0,1953,0,09302016
7,20.0,101.0,101.0,NJ,57,0,0,1959,0,09302016
8,21.0,101.0,101.0,NY,46,0,0,1970,0,09302016
9,22.0,101.0,101.0,NY,48,0,0,1968,0,09302016


In [18]:
dim_personal_data.dtypes

cicid             float64
citizenship       float64
residency         float64
state_code         object
age                 int64
gender             object
insnum             object
birth_year          int64
occupation         object
admission_date     object
dtype: object

Using Pyspark to work on I94 dataset

In [ ]:
#from pyspark.sql import SparkSession

#spark = SparkSession.builder.\
#config("spark.jars.repositories", "https://repos.spark-packages.org/").\
#config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
#enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

The second dataset <strong>Global Temperature Dataset</strong> explore, clean and break it into dimension table.

In [19]:
#Exploring the Local Weather Temperature
fname_temp = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname_temp)

In [20]:
df_temperature.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
9,1744-08-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [21]:
df_temperature.shape #the world temp data has 8 million rows and when we take out only for US

(8599212, 7)

In [22]:
def clean_df_temperature(df_temperature):
    dim_temperature = df_temperature.copy()
    dim_temperature = dim_temperature[dim_temperature['Country']=='United States']
    dim_temperature = dim_temperature.drop('Latitude', 1)
    dim_temperature = dim_temperature.drop('Longitude', 1)
    dim_temperature['dt'] = pd.to_datetime(dim_temperature['dt']).dt.date
    return dim_temperature

In [23]:
dim_temperature = clean_df_temperature(df_temperature)

In [31]:
#dim_temperature = df_temperature.copy()

#dim_temperature = dim_temperature[dim_temperature['Country']=='United States']

#dim_temperature = dim_temperature.drop('Latitude', 1)
#dim_temperature = dim_temperature.drop('Longitude', 1)

In [24]:
dim_temperature.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country
47555,1820-01-01,2.101,3.217,Abilene,United States
47556,1820-02-01,6.926,2.853,Abilene,United States
47557,1820-03-01,10.767,2.395,Abilene,United States
47558,1820-04-01,17.989,2.202,Abilene,United States
47559,1820-05-01,21.809,2.036,Abilene,United States
47560,1820-06-01,25.682,2.008,Abilene,United States
47561,1820-07-01,26.268,1.802,Abilene,United States
47562,1820-08-01,25.048,1.895,Abilene,United States
47563,1820-09-01,22.435,2.216,Abilene,United States
47564,1820-10-01,15.830,2.169,Abilene,United States


In [23]:
dim_temperature.shape

(687289, 5)

In [31]:
dim_temperature.isna().sum()

dt                                   0
AverageTemperature               25765
AverageTemperatureUncertainty    25765
City                                 0
Country                              0
dtype: int64

In [32]:
dim_temperature.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
dtype: object

In [25]:
dim_temperature['dt'] = pd.to_datetime(dim_temperature['dt']).dt.date

In [34]:
dim_temperature.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
dtype: object

In [35]:
dim_temperature['City'].value_counts()

Springfield         9545
Columbus            6478
Aurora              6101
Arlington           5564
Peoria              5384
Richmond            5216
Pasadena            4302
Glendale            4122
Chattanooga         3239
Syracuse            3239
Evansville          3239
Saint Louis         3239
South Bend          3239
Fort Wayne          3239
Rochester           3239
Stamford            3239
Hampton             3239
Saint Petersburg    3239
Madison             3239
Edison              3239
New York            3239
Jacksonville        3239
New Haven           3239
Winston Salem       3239
Bridgeport          3239
Atlanta             3239
Manchester          3239
Chicago             3239
Charlotte           3239
Washington          3239
                    ... 
Anaheim             1977
Santa Clara         1977
Corona              1977
Simi Valley         1977
Salinas             1977
Fairfield           1977
Sacramento          1977
Antioch             1977
Burbank             1977


The third dataset <strong>US cities demographics</strong> Dataset explore, clean and break it into dimension table.

In [26]:
dim_demographics = pd.read_csv("./us-cities-demographics.csv", delimiter=";")

In [27]:
dim_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [28]:
dim_demographics.shape

(2891, 12)

In [39]:
dim_demographics.dtypes

City                       object
State                      object
Median Age                float64
Male Population           float64
Female Population         float64
Total Population            int64
Number of Veterans        float64
Foreign-born              float64
Average Household Size    float64
State Code                 object
Race                       object
Count                       int64
dtype: object

In [29]:
dim_demographics.rename(columns={'Median Age': 'median_age', 'Male Population': 'male_population','Female Population':'female_population','Total Population':'total_population','Number of Veterans':'no_of_veterans','Average Household Size':'household_size','State Code':'state_code', 'Foreign-born': 'foreign_born'}, inplace=True)

In [30]:
dim_demographics.head()

,City,State,median_age,male_population,female_population,total_population,no_of_veterans,foreign_born,household_size,state_code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data:
 - Changed SAS timestamp to datetime 
 - Changed certain float columns to int types
 - Changed column names to more descriptive names
 - For the temperature dataset only took out for Country United States since we are dealing with only US states here.
 - Added country to immigration dataframe to be able to merge on the country column for analysis.
 - Dropped coulmns that proved not be helpful.

In [66]:
# Performing cleaning tasks here
#completed in the above steps.




### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The Conceptual data model is attached as a png file. I am using a Star Schema. A fact_immigration table as the FACT Table and 
dim_airline, dim_temperature, dim_demographics and dim_personal_info as the dimensional tables.

When thinking about the data model, I was thinking about creating one table that has the facts on the immigration data and other table that has more details that could help answer my questions, so the Fact table for the facts and the dimension Table for the details for the analysis. Why I chose Star schema over snowflake schema is because 1. Star schema is more efficient and a better way to organize the data in the dataware house. and 2. Because they are superfact, simple design that I was looking for to combine my fact and dimension tables.
#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
Creak the df_immigration table to fact_immigration, df_airline and df_personal_info. From the immigration table we the Facts on the immigration, the personal info and the aurline details. The tenperature and the demographics table are used as it is for dimensions.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
First create the db and tables by running python create_tables.py in the terminal because running the below.

In [31]:
fact_immigration = fact_immigration[:200]
fact_immigration.shape

(200, 10)

In [32]:
dim_airline_details = dim_airline_details[:200]
dim_airline_details.shape

(200, 5)

In [33]:
dim_personal_data = dim_personal_data[:200]
dim_personal_data.shape

(200, 10)

In [34]:
dim_demographics = dim_demographics[:200]
dim_demographics.shape

(200, 12)

In [35]:
dim_temperature = dim_temperature[:200]

In [36]:
import psycopg2
from sql_queries import fact_immigration_table_insert, dim_airline_table_insert, dim_temperature_table_insert, dim_demographics_table_insert, dim_personal_info_table_insert 

In [37]:
# Write code here
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [38]:
for index, row in fact_immigration.iterrows():
    cur.execute(fact_immigration_table_insert, list(row.values))
    conn.commit()

In [39]:
for index, row in dim_airline_details.iterrows():
    cur.execute(dim_airline_table_insert, list(row.values))
    conn.commit()

In [40]:
for index, row in dim_personal_data.iterrows():
    cur.execute(dim_personal_info_table_insert, list(row.values))
    conn.commit()

In [41]:
for index, row in dim_demographics.iterrows():
    cur.execute(dim_demographics_table_insert, list(row.values))
    conn.commit()

In [42]:
for index, row in dim_temperature.iterrows():
    cur.execute(dim_temperature_table_insert, list(row.values))
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database on making sure the primary key is not duplicate.
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [43]:
# Perform quality checks here
cur.execute("SELECT COUNT(*) FROM fact_immigration")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table fact_immigration")

cur.execute("SELECT COUNT(*) FROM dim_airline")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table dim_airline")

In [44]:
cur.execute("SELECT cicid, COUNT(cicid) FROM fact_immigration GROUP BY cicid HAVING COUNT(cicid) > 1")
conn.commit()
print(cur.rowcount)

0


In [45]:
cur.execute("SELECT cicid, count(cicid) FROM fact_immigration f \
             INNER JOIN dim_demographics d ON f.state_code = d.state_code \
             GROUP BY 1")
conn.commit()
print(cur.rowcount) #printitng the rowcount.The ETL is working, checked with mergeing fact_immigration and dim_demographics table.

196


Showcasing the answer to my question that I wrote in the beginning for why I was looking at this data:
 - immigrants move to US cities with what kind of demographics statistics?
 - immigrants move to US cities with what kind of temperature, do they move to warmer weather or cooler weather?
 - Do they move to cities with a larger population or less population?

In [46]:
cur.execute("SELECT cicid, count(cicid), d.median_age, d.male_population, d.female_population FROM fact_immigration f \
             INNER JOIN dim_demographics d ON f.state_code = d.state_code \
             GROUP BY 1,3,4,5")
conn.commit()
print(cur.rowcount)

1081


In [47]:
#immigrants move to US cities with what kind of demographics statistics?
for row in cur:
    print(row)

(89.0, 1, 36.6, 66157.0, 70958.0)
(153.0, 1, 33.1, 124537.0, 133529.0)
(181.0, 1, 36.1, 33146.0, 36336.0)
(196.0, 1, 33.8, 99391.0, 100260.0)
(88.0, 1, 41.4, 37155.0, 42614.0)
(100.0, 1, 31.6, 57326.0, 60784.0)
(178.0, 1, 41.6, 69515.0, 76943.0)
(188.0, 1, 28.7, 38614.0, 41198.0)
(163.0, 1, 41.4, 37155.0, 42614.0)
(164.0, 3, 47.0, 55225.0, 57754.0)
(205.0, 1, 35.4, 45246.0, 47688.0)
(34.0, 1, 29.9, 63765.0, 66545.0)
(201.0, 2, 28.8, 124305.0, 131484.0)
(104.0, 4, 35.3, 48984.0, 49313.0)
(87.0, 1, 38.3, 42926.0, 45049.0)
(176.0, 1, 42.5, 48090.0, 44221.0)
(22.0, 1, 33.1, 124537.0, 133529.0)
(194.0, 1, 44.8, 65113.0, 64216.0)
(71.0, 1, 42.5, 48090.0, 44221.0)
(194.0, 1, 35.0, 1958998.0, 2012898.0)
(70.0, 1, 36.6, 66157.0, 70958.0)
(201.0, 1, 29.4, 41761.0, 40345.0)
(99.0, 1, 33.1, 90811.0, 96944.0)
(194.0, 1, 34.2, 54407.0, 57619.0)
(33.0, 2, 34.5, 116406.0, 120430.0)
(227.0, 1, 33.1, 124537.0, 133529.0)
(83.0, 1, 31.0, 41028.0, 39306.0)
(55.0, 1, 38.0, 96580.0, 104538.0)
(69.0, 1, 33.5,

In [50]:
cur.execute("SELECT d.total_population FROM fact_immigration f \
             INNER JOIN dim_demographics d ON f.state_code = d.state_code \
             GROUP BY 1")
conn.commit()
print(cur.rowcount)

123


In [51]:
#Do they move to cities with a larger population or less population?: Total population of the cities the immigrants moved into listed.
for row in cur:
    print(row)

(107771,)
(76996,)
(136454,)
(72096,)
(83867,)
(304385,)
(655760,)
(130310,)
(71028,)
(463875,)
(189894,)
(87750,)
(257088,)
(113117,)
(836969,)
(258066,)
(92934,)
(89746,)
(672228,)
(373627,)
(96098,)
(90711,)
(84839,)
(146458,)
(87093,)
(88161,)
(161113,)
(145124,)
(86086,)
(654596,)
(110402,)
(132342,)
(520072,)
(88809,)
(108489,)
(83339,)
(681136,)
(82463,)
(236836,)
(140809,)
(175232,)
(76131,)
(76368,)
(71567,)
(98297,)
(84662,)
(403091,)
(187755,)
(85032,)
(67106,)
(72809,)
(96794,)
(126216,)
(149346,)
(110223,)
(68801,)
(103467,)
(324082,)
(255789,)
(116745,)
(87975,)
(78295,)
(79769,)
(104709,)
(152589,)
(93629,)
(1567442,)
(96659,)
(112979,)
(80334,)
(92311,)
(71024,)
(120971,)
(106782,)
(71026,)
(234793,)
(157386,)
(79812,)
(94633,)
(83276,)
(149190,)
(199651,)
(203715,)
(110827,)
(71210,)
(69482,)
(92459,)
(189114,)
(74015,)
(350738,)
(129329,)
(132062,)
(682545,)
(69947,)
(113199,)
(128877,)
(72856,)
(118110,)
(72005,)
(281913,)
(193955,)
(109717,)
(85264,)
(85599,)
(94496

In [52]:
cur.execute("SELECT d.AverageTemperature FROM fact_immigration f \
             INNER JOIN dim_temperature d ON f.Country = d.Country \
             GROUP BY 1")
conn.commit()
print(cur.rowcount)

198


In [53]:
#Average Temperarures of the cities that immigrants moved to are listed.
for row in cur:
    print(row)

(3.977,)
(11.728,)
(26.646,)
(26.541,)
(16.462,)
(9.993,)
(3.264,)
(22.045,)
(28.631,)
(28.04,)
(21.809,)
(22.365,)
(28.054,)
(14.743,)
(8.169,)
(15.461,)
(16.64,)
(13.327,)
(11.923,)
(27.562,)
(21.317,)
(12.575,)
(6.86,)
(25.768,)
(21.593,)
(4.964,)
(22.689,)
(26.268,)
(6.021,)
(21.092,)
(9.522,)
(16.918,)
(26.553,)
(26.943,)
(22.217,)
(25.836,)
(5.815,)
(23.164,)
(5.296,)
(27.031,)
(11.491,)
(6.517,)
(4.287,)
(8.366,)
(27.123,)
(6.439,)
(6.926,)
(22.198,)
(7.241,)
(27.124,)
(25.699,)
(10.096,)
(15.837,)
(21.503,)
(25.682,)
(13.302,)
(6.564,)
(16.722,)
(15.738,)
(7.17,)
(6.274,)
(16.133,)
(17.041,)
(28.205,)
(27.272,)
(12.18,)
(7.332,)
(21.491,)
(12.227,)
(27.444,)
(27.133,)
(21.488,)
(12.561,)
(21.406,)
(26.208,)
(21.288,)
(29.641,)
(25.612,)
(4.117,)
(21.034,)
(21.26,)
(10.767,)
(15.83,)
(5.353,)
(15.643,)
(11.241,)
(27.189,)
(6.877,)
(21.65,)
(6.417,)
(17.155,)
(28.598,)
(3.737,)
(17.663,)
(22.435,)
(11.126,)
(15.969,)
(17.224,)
(3.853,)
(22.809,)
(21.547,)
(17.195,)
(20.924,)
(21.

I was able ot answer the questions I had asked when I saw the data set. The ETL that was created helped me answer the questions by cimbing the fact and the dimension tables.

In [54]:
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.
	dim_airline	
Stores information on the airline		
cicid	Primary key for the i94 data	
airline	airline the passenger flew in	
admnum	admission number of the i94 immigrant	
fltno	Flight number they arrived in	
visatype	the type of visa that was issued	


dim_temperarure				
Stores details on the temperature of the US Cities				
dt 	datetime when data was captured			
AverageTemperature	the average temperature			
AverageTemperatureUncertainty	the average Temperature Uncertainity			
City 	The city the data was temperatured was captured for			
Country	The Country is United States			

For dim_demographics and dim_personal_info table the names of the columns are self explanatory


fact_immigration- stores fact about the immigration table		
cicid	The primary key for the i94 immigrant data	
year	year of the arrival of passengers	
month	month of the arrival of passengers	
port	which i94 port the immigrants arrived in	
arrival_date		
departure_date		
visa	type of visa they arrived on	
mode	which mode: sea, air, land	
state_code	which state they arrived to	
Country	Country is United States	

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
    - The data should be updated monthly. Because we are basing the fact table for per month bases.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x. - We would need to use a GPU to process data efficiently, Would need to use more efficient tools like EMR for data analysis. We will still use PySpark to analyze the data.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day. - Use Airflow DAG to schedule daily upload to the dashboard.
 * The database needed to be accessed by 100+ people. Using cloud services like AWS Redshift the data can be accessed by 100+ people.